In [150]:
# def getImages(url, fileName):
#     import os
#     import selenium 
#     from selenium import webdriver
    
#     driver = webdriver.Firefox(executable_path="C:/Users/Farzad/webDrivers/geckodriver.exe")
    
#     import urllib.request
#     import os
    
    
#     #os.chdir(str(directory)+'/'+str(fileName))
#     #retval = os.getcwd()
#     #print(retval)
#     #print()
    
#     driver.get(url)
    
#     #ivLargeImage
#     get_images = driver.find_element_by_id('ivLargeImage')
#     get_images = driver.find_element_by_id('altImages')
#     images = get_images.find_elements_by_tag_name('img')
    
#     image_list = []
#     for each_image in images:
#         image_list.append(each_image.get_attribute('src'))
    
#     HQ_images= []
#     for k in range (1, int(len(image_list))):
#         HQ_images.append(image_list[k].replace("US40","SL1500"))
    
#     for i in range (0, int(len(HQ_images))):

#         titlePrefix= str(i+1)+str("_")
#         try:
#             print("Image number_" + str(i+1))
#             print("Downloading starts...\n")
#             urllib.request.urlretrieve(str(HQ_images[i]), str(titlePrefix) + str(fileName)+str('.jpg'))
#             print(str((HQ_images[i])))
#             print("Download completed..!!")
#             print()
#         except Exception as e:
#             print(e)

#     print("All files Downloaded")
    
#     #driver.close()
#     driver.quit()






# # To download images in folders in a loop
# #csvfilePath is the path of productUrl.csv file
# #directory is the path of in which you want to store folders of images
# def getImages_loop(csvfilePath, directory ):
#     import pandas
#     import time
#     import os

#     raw_dataUrl = pandas.read_csv(csvfilePath)
#     dataUrl = raw_dataUrl['raw']
#     os.chdir(directory)

#     for i in range (0, int(len(dataUrl))):
#     #for i in range (0, 1):

#         fileName= 'prd_'+str(i+1)
#         os.mkdir(fileName) # Make a folder
#         os.chdir(fileName) #Change directory to new folder
#         url = dataUrl[i]
#         # download images
#         try:
#             getImages(url, fileName)
#         except:
#             print('getImages failed for: '+str(i))
#             pass

#         #parent = os.path.abspath('..')
        
        
#         os.chdir(directory) # change directory to original path

#         print('download completed for fileName: ' + str(fileName))
        
#         time.sleep(10)

In [151]:
def getProductData(url):
    import os
    import selenium 
    from selenium import webdriver
    
    driver = webdriver.Firefox(executable_path="C:/Users/Farzad/webDrivers/geckodriver.exe")
    driver.get(url)
    
    singleProduct=[]
    
    try:
        get_title = driver.find_elements_by_class_name("product-title-word-break")
        title = get_title[0].text
    except: pass
        
    try: 
    #feature-bullets
        get_des = driver.find_elements_by_id("feature-bullets")
        strDes = str(get_des[0].text.split('\n'))
        des= strDes.replace("]", "").replace("[", "")
    except: pass

    try:
        #a-keyvalue prodDetTable
        get_details = driver.find_elements_by_class_name("prodDetTable")
        strDetails = str(get_details[0].text.split('\n'))
        details= strDetails.replace("]", "").replace("[", "")
    except: pass
    
    try:
    #productDescription
        get_long_des = driver.find_element_by_id('productDescription')
        strLong_des = str(get_long_des.text.split('\n'))
        long_des= strLong_des.replace("]", "").replace("[", "")
        
    except: pass
    try: 
        get_availability = driver.find_element_by_id('availability')
        availability = get_availability.text.split('\n')
        
    except: pass
        
    try:
        get_brand = driver.find_element_by_id('bylineInfo')
        strBrand= str(get_brand.text.split('\n'))
        brand= strBrand[9:].replace("]", "").replace("'", "")
    except: pass

    try:
    #buybox-tabular-content
        get_sellerDetails = driver.find_elements_by_class_name("buybox-tabular-content")
    except: pass
    
    try:
        shipsFrom= get_sellerDetails[0].text
    except: pass
        
    try:
        soldBy= get_sellerDetails[1].text
    except: pass
    #priceblock_ourprice
    try:
        get_price = driver.find_element_by_id('priceblock_ourprice')
        price = get_price.text
    except: pass
    #return title, brand, des, details, long_des, availability, shipsFrom, soldBy, price, url

    d={}

    
    try: d['title']=title
    except:d['title']=None
    try: d['brand']= brand
    except:d['brand']=None
    try: d['description']= des
    except:d['description']=None
    try: d['details']= details
    except:d['details']=None
    try: d['long description']=long_des
    except:d['long description']=None
    try: d['availability']= availability
    except:d['availability']=None
    try: d['ships from']= shipsFrom
    except:d['ships from']=None
    try: d['sold by']=soldBy
    except:d['sold by']=None
    try: d['price']= price
    except:d['price']=None
    try: d['product url']= url
    except:d['product url']=None

    singleProduct.append(d)

    import pandas
    
    df_mini = pandas.DataFrame(singleProduct)
    
    driver.close()
    driver.quit()
    
    return df_mini







def getProductData_Loop(directory, csvfilePath, fileName):
    
    import pandas
    import time
    import os

    #raw_dataUrl = pandas.read_csv("D:/Ebay/aus/oct/productsUrl.csv")
    raw_dataUrl = pandas.read_csv(csvfilePath)
    dataUrl = raw_dataUrl['raw']
    #directory= 'D:/Ebay/aus/oct/'
    os.chdir(directory)

    l=getProductData(dataUrl[0])
    
    
    for i in range(1, len(dataUrl)):
    #for i in range(1, 3):

        try: 
            url = dataUrl[i]

            l= l.append(getProductData(dataUrl[i]))
    
            
        except:
            print('Failed getProductData_Loop for: ' +(str(i)))
            pass

        time.sleep(10)
        
    df= l.to_csv(fileName + str('.csv'))
        
    location= os.getcwd()
        
    return l.shape, print('csv File is saved at:' + str(location)) 
    
    

In [152]:
directory= input('directory :')
csvfilePath= input('csvfilePath :')
fileName = input('fileName: ')

getProductData_Loop(directory, csvfilePath, fileName)

directory :D:/Ebay/aus/oct/
csvfilePath :D:/Ebay/aus/oct/productsUrl.csv
fileName: data_v2
csv File is saved at:D:\Ebay\aus\oct


((30, 10), None)

In [1]:
# directory= input('directory :')
# csvfilePath= input('csvfilePath :')


# getImages_loop(csvfilePath, directory )

In [ ]:
D:/Ebay/aus/oct/
D:/Ebay/aus/oct/productsUrl.csv
test